In [1]:
import sys
sys.path.append("/Users/mocquin/MYLIB10/MODULES/noise3D/")

In [2]:
import numpy as np

import noise3d

# We'll be creating 3D sequences with T=100 frames, V=100 lines, and H=100 columns.
T = 100
V = 100
H = 100
# but you can define any shape

# Create some seq to apply operators on
seq = noise3d.genseq.genseq_3dnoise_seq(T, V, H, [1 for i in range(7)], [0 for i in range(7)])

# Basic noise variances

In [3]:
# Getting the individual noise variances 
print(noise3d.noise.var_nt(seq))
print(noise3d.noise.var_nv(seq))
print(noise3d.noise.var_nh(seq))
print(noise3d.noise.var_ntv(seq))
print(noise3d.noise.var_nth(seq))
print(noise3d.noise.var_nvh(seq))
print(noise3d.noise.var_ntvh(seq))
print()

# Wrapping all in a function, as well as total variance
noise_vars = noise3d.noise.get_all_3d_noise_var(seq, names=True)
for var, name in zip(noise_vars[:-1], noise_vars[-1]):
    print(f"Var({name:3}) : {var:.2f}")



0.9733998628983506
0.9416037747588615
0.7729349087086768
0.9902274395502355
1.005981064689297
0.9853919223716192
0.9715771601242433

Var(t  ) : 0.97
Var(v  ) : 0.94
Var(h  ) : 0.77
Var(tv ) : 0.99
Var(th ) : 1.01
Var(vh ) : 0.99
Var(tvh) : 0.97
Var(tot) : 6.61


## Faster equivalent version

A faster equivalent version 

In [4]:
noise_vars = noise3d.noise.get_all_3d_noise_var_fast(seq, names=True)
for var, name in zip(noise_vars[:-1], noise_vars[-1]):
    print(f"Var({name:3}) : {var:.2f}")


Var(t  ) : 0.97
Var(v  ) : 0.94
Var(h  ) : 0.77
Var(tv ) : 0.99
Var(th ) : 1.01
Var(vh ) : 0.99
Var(tvh) : 0.94
Var(tot) : 6.61


In [5]:
%timeit noise3d.noise.get_all_3d_noise_var_fast(seq)
%timeit noise3d.noise.get_all_3d_noise_var(seq)

5.74 ms ± 112 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
42.7 ms ± 1.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
